In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from utils.dataset import SICKLE_Dataset

/home/dlpl/miniconda3/envs/sickle/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils import utae_utils, model_utils
from torch.utils import data
import torch

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv("dataset/sickle_dataset_tabular.csv")

In [5]:
train_df = df[df.SPLIT == "train"].reset_index(drop=True)
val_df = df[df.SPLIT == "val"].reset_index(drop=True)
test_df = df[df.SPLIT == "test"].reset_index(drop=True)

In [6]:
# Configuration dictionary for dataset
dataset_config = {
    'data_dir': 'dataset_20_per/',
    'satellites': {
        "S2": {
        "bands": ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'],
        "rgb_bands": [3, 2, 1],
        "mask_res": 10,
        "img_size": (32, 32),
        }
    },
    'ignore_index': -999,
    'transform': None,
    'actual_season': False,
    'phase': 'eval'  # Assuming phase could be 'train' or 'eval' based on your setup
}

In [7]:
dt_train = SICKLE_Dataset(df= train_df, **dataset_config)

In [8]:
print(dt_train)

In [9]:
dt_val = SICKLE_Dataset(df= val_df, **dataset_config)

In [10]:
np.set_printoptions(threshold=np.inf)

In [11]:
print(dt_train[0][0])

{'S2': (array([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.

In [12]:
print(len(dt_train))
print(len(dt_val))

1937
227


In [13]:
#  WRITTEN RANDOM VALUES HERE TO SEE IF CODE IS WORKING
batch_size = 227
num_workers = 5

In [14]:
collate_fn = lambda x: utae_utils.pad_collate(x, pad_value=0)
train_loader = data.DataLoader(
    dt_train,
    batch_size=1937,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
)
val_loader = data.DataLoader(
    dt_val,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=num_workers,
)

In [15]:
batch_data, masks = next(iter(train_loader))

/home/dlpl/code/SICKLE/utils/utae_utils.py:37: UserWarning: An output with one or more elements was resized since it had shape [690253824], which does not match the required output shape [1937, 29, 12, 32, 32]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /opt/conda/conda-bld/pytorch_1666643016022/work/aten/src/ATen/native/Resize.cpp:17.)
  return torch.stack(batch, 0, out=out)
/home/dlpl/code/SICKLE/utils/utae_utils.py:37: UserWarning: An output with one or more elements was resized since it had shape [56173], which does not match the required output shape [1937, 29]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0)

In [16]:
val_batch_data, val_masks = next(iter(val_loader))

/home/dlpl/code/SICKLE/utils/utae_utils.py:37: UserWarning: An output with one or more elements was resized since it had shape [72523776], which does not match the required output shape [227, 26, 12, 32, 32]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /opt/conda/conda-bld/pytorch_1666643016022/work/aten/src/ATen/native/Resize.cpp:17.)
  return torch.stack(batch, 0, out=out)
/home/dlpl/code/SICKLE/utils/utae_utils.py:37: UserWarning: An output with one or more elements was resized since it had shape [5902], which does not match the required output shape [227, 26]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (T

In [17]:
print(len(val_masks['crop_yield']))

227


In [18]:
task= "crop_yield"

In [19]:
for sat in dataset_config['satellites'].keys():
        (samples, dates) = batch_data[sat]
        print(f"-----------{sat}------------")
        print("Samples Shape", samples.shape, "Masks Shape", masks["crop_type"].shape)
        print("dates", dates[0])
        print("Samples", torch.unique(samples[0]))
        print("Masks", torch.unique(masks[task]))

-----------S2------------
Samples Shape torch.Size([1937, 29, 12, 32, 32]) Masks Shape torch.Size([1937, 32, 32])
dates tensor([ 15,  60,  80, 115, 130, 145,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0])
Samples tensor([    0.,   115.,   127.,  ..., 16053., 16054., 16055.])
Masks tensor([-999.0000,   24.0000,   28.2759,   28.9286,   32.1429,   33.3333,
          37.2727,   37.3333,   37.5000,   38.6667,   40.0000,   40.4444,
          41.0256,   41.5385,   42.6316,   43.0769,   44.1379,   45.0000,
          45.5172,   45.7895,   46.1538,   47.0270,   47.5676,   48.0000,
          48.2759,   49.2308,   50.0000,   51.1111,   51.6667,   53.3333,
          54.3750,   54.6667,   55.6522,   56.2500,   56.5517,   57.0000,
          58.0645,   58.5000,   59.1304,   59.3333,   60.0000,   61.5385,
          61.9355,   63.5294,   63.8710,   64.0000,   64.1667,   64.8000,
          66.6667,   67.2000,   67.5000, 

In [20]:
for i in range(100):
    a = torch.unique(masks[task][i])
    print(a)

tensor([-999.], dtype=torch.float64)
tensor([-999.0000,   28.2759,   61.5385,   70.0000,   70.8333,  136.6667],
       dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.0000,  141.4286,  220.0000,  274.2857], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.0000,  137.1429,  160.0000,  265.7143], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-999.], dtype=torch.float64)
tensor([-

In [21]:
# print(batch_data)
print(batch_data)
# save this dictionary to a txt file
# with open('batch_data.txt', 'w') as file:
#      file.write(str(batch_data))

{'S2': [tensor([[[[[  220.,   220.,   220.,  ...,   347.,   347.,   347.],
           [  220.,   220.,   220.,  ...,   347.,   347.,   347.],
           [  220.,   220.,   220.,  ...,   347.,   347.,   347.],
           ...,
           [  765.,   765.,   765.,  ...,   262.,   262.,   262.],
           [  765.,   765.,   765.,  ...,   262.,   262.,   262.],
           [  765.,   765.,   765.,  ...,   262.,   262.,   262.]],

          [[  417.,   376.,   339.,  ...,   313.,   252.,   284.],
           [  351.,   321.,   279.,  ...,   312.,   310.,   285.],
           [  252.,   245.,   249.,  ...,   327.,   299.,   291.],
           ...,
           [ 1282.,  1394.,  1362.,  ...,   278.,   327.,   336.],
           [ 1276.,  1380.,  1340.,  ...,   262.,   290.,   320.],
           [ 1318.,  1356.,  1406.,  ...,   283.,   304.,   293.]],

          [[  733.,   689.,   727.,  ...,   647.,   540.,   516.],
           [  695.,   672.,   620.,  ...,   645.,   601.,   579.],
           [  639.

In [22]:
print(masks['crop_yield'])

tensor([[[-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.],
         ...,
         [-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.]],

        [[-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.],
         ...,
         [-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.]],

        [[-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.],
         [-999., -999., -999.,  ..., -999., -999., -999.],
         ...,
         [-999., -999., -999.,  ..., -999., -999., -999.],
         [

In [23]:
print("Original batch_data['S2'][0] shape:", batch_data['S2'][0].shape)  
print("Original masks['crop_yield'] shape:", masks['crop_yield'].shape)  

Original batch_data['S2'][0] shape: torch.Size([1937, 29, 12, 32, 32])
Original masks['crop_yield'] shape: torch.Size([1937, 32, 32])


In [24]:
print("Original batch_data['S2'][0] shape:", val_batch_data['S2'][0].shape)  
print("Original masks['crop_yield'] shape:", val_masks['crop_yield'].shape)  

Original batch_data['S2'][0] shape: torch.Size([227, 26, 12, 32, 32])
Original masks['crop_yield'] shape: torch.Size([227, 32, 32])


In [25]:
import torch

# Assuming 'batch_data' is your loaded batch data with shape [100, 74, 12, 32, 32]
# Indices for Red and NIR bands (assuming index starts at 0 and Band 4 is at index 3 and Band 8 is at index 7)
red_band_index = 3
nir_band_index = 7

# Extracting the Red and NIR bands
red = batch_data['S2'][0][:, :, red_band_index, :, :]  # shape [100, 74, 32, 32]
nir = batch_data['S2'][0][:, :, nir_band_index, :, :]  # shape [100, 74, 32, 32]

# Calculate NDVI
ndvi = (nir - red) / (nir + red + 1e-10)  # Adding a small number to avoid division by zero

# Example: Flatten NDVI to use as features, you may choose to reshape or summarize differently based on your needs
ndvi_flat = ndvi.reshape(ndvi.shape[0], -1)  # Flattening each image in the batch

print("NDVI calculated with shape:", ndvi.shape)

NDVI calculated with shape: torch.Size([1937, 29, 32, 32])


In [26]:
# FOR VAL

# Extracting the Red and NIR bands
val_red = val_batch_data['S2'][0][:, :, red_band_index, :, :]  # shape [100, 74, 32, 32]
val_nir = val_batch_data['S2'][0][:, :, nir_band_index, :, :]  # shape [100, 74, 32, 32]

# Calculate NDVI
val_ndvi = (val_nir - val_red) / (val_nir + val_red + 1e-10)  # Adding a small number to avoid division by zero

# Example: Flatten NDVI to use as features, you may choose to reshape or summarize differently based on your needs
val_ndvi_flat = val_ndvi.reshape(val_ndvi.shape[0], -1)  # Flattening each image in the batch

print("NDVI calculated with shape:", val_ndvi.shape)

NDVI calculated with shape: torch.Size([227, 26, 32, 32])


In [27]:
print(ndvi_flat.shape)

torch.Size([1937, 29696])


In [28]:
# Temporal aggregation - using the mean as an example
ndvi_mean = torch.mean(ndvi, dim=1)  # Now shape [100, 32, 32], mean over temporal dimension

In [29]:
# Assuming ndvi_mean is correctly shaped as [100, 32, 32]
X_train = ndvi_mean.reshape(-1, 32*32)  # This now correctly reshapes to [100, 1024] as intended
y_train = masks['crop_yield'].reshape(-1)  # Flatten masks to [100 * 1024]

# Since each pixel across each image in the batch is now a sample:
X_train = X_train.reshape(-1, 1)  # Reshape X_train to [100*1024, 1] where each row is a pixel feature vector
y_train = y_train.flatten()  # Ensure y_train is a flat array [100*1024]

# Now X_train and y_train should be aligned:
print("X_train shape:", X_train.shape)  # Should be (102400, 1)
print("y_train shape:", y_train.shape)  # Should be (102400,)

# Filter out invalid data points where y_train is -999
valid_indices = y_train != -999
X_train = X_train[valid_indices]
y_train = y_train[valid_indices]

X_train shape: torch.Size([1983488, 1])
y_train shape: torch.Size([1983488])


In [30]:
# Proceed to train the RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [31]:
# Proceed to train the RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
rf_model_l8 = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_l8.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [32]:
# Proceed to train the RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
rf_model_s1 = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_s1.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [33]:
# Assuming val_ndvi and val_masks are available
# Temporal aggregation (mean NDVI over temporal dimension)
val_pri_mean = torch.mean(val_ndvi, dim=1)  # [batch_size, height, width]

# Reshape validation NDVI to match the training data's shape
X_val = val_pri_mean.reshape(-1, 32*32)  # Flatten to [batch_size, features]
y_val = val_masks['crop_yield'].reshape(-1)  # Flatten masks to [batch_size * height * width]
print(X_val.shape)
print(y_val.shape)


torch.Size([227, 1024])
torch.Size([232448])


In [34]:
# Reshape X_val to have one pixel per row, matching the y_val structure
X_val = X_val.reshape(-1, 1)  # This should result in [100*1024, 1], i.e., [102400, 1]
print("Reshaped X_val:", X_val.shape)
print("y_val shape:", y_val.shape)

# Proceed with filtering out invalid data points where y_val is -999
valid_indices_val = y_val != -999
X_val = X_val[valid_indices_val]
y_val = y_val[valid_indices_val]

Reshaped X_val: torch.Size([232448, 1])
y_val shape: torch.Size([232448])


In [59]:
# Predict with the trained RandomForest model
y_pred_s2 = np.load('y_pred_s2_20.npy')

In [60]:
print(y_pred_s2.shape)

(773,)


In [55]:

# Predict with the trained RandomForest model
y_pred_l8 = np.load('y_pred_l8_20.npy')


In [57]:
print(y_pred_l8.shape)

(773,)


In [37]:
y_pred_s1 = rf_model_s1.predict(X_val)

In [58]:
print(y_pred_s1.shape)

(773,)


In [61]:
y_pred_combined = (y_pred_s2 + y_pred_l8 + y_pred_s1) / 3


In [40]:
import numpy as np
from sklearn.metrics import accuracy_score

# Assuming y_pred and y_val are continuous and need to be categorized for demonstration
# Example thresholds for categorization could be set based on the distribution of y_val
thresholds = [np.percentile(y_val, 33), np.percentile(y_val, 66)]  # Divide into tertiles
y_val_cat = np.digitize(y_val, bins=thresholds)
y_pred_cat = np.digitize(y_pred, bins=thresholds)

# Now calculate accuracy
accuracy = accuracy_score(y_val_cat, y_pred_cat)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.35446313065976714


In [63]:
import numpy as np

def calculate_mape(y_true, y_pred):
    """ Calculate MAPE avoiding division by zero and convert to percentage """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0  # Ensure we don't divide by zero
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100



# Calculate MAPE
mape = calculate_mape(y_val, y_pred_combined)
print("Validation MAPE:", mape)


Validation MAPE: 82.18479233015657


Dataset Exploration